<a href="https://colab.research.google.com/github/MudassirABBASSi/Computer-Vision/blob/main/Yolo_(object_detection_library).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configure API keys
To fine-tune YOLO11, you need to provide your Roboflow API key. Follow these steps:

Go to your Roboflow Settings page. Click Copy. This will place your private key in the clipboard.
In Colab, go to the left pane and click on Secrets (🔑). Store Roboflow API Key under the name ROBOFLOW_API_KEY.

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found
